# Imports

In [1]:
from numpy import *
from numpy import pi as π
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interactive

# Ideal case

Consideremos que a fonte de luz seja um laser com um único modo transversal que se propaga como ondas paraxiais (ou seja, cujas normais às frentes de onda sejam raios paraxiais), mais especificamente do tipo **feixe gaussiano**.

## Conceito de raios paraxiais

Em meios isotrópicos,raios ópticos apontam na direção do fluxo de energia óptica. Um conjunto de raios pode ser construído no qual a densidade de raios é proporcional a densidade energia luminosa. Raios podem ser traçados através de um sistema óptico para se determinar a energia óptica atravessando uma dada área. Componentes ópticos são comumente alinhados com um eixo óptico, em torno do qual os raios viajam com inclinações pequenas. Tais raios são chamados **raios paraxiais**. O conceito de raios advém da óptica geométrica, do qual relembraremos um dos postulados, conhecido como **princípio de Fermat**:

"A luz viaja ao longo do caminho de menor tempo"

matematicamente expresso como

\begin{equation}
\delta\int_A^B n(\mathbf{r}) ds = 0
\end{equation}

sendo $n$ o índice de refração na posição $\mathbf r$ (razão entre a velocidade da luz no vácuo e a velocidade da luz no meio) e $ds$ um elemento diferencial de comprimento ao longo do caminho de porpagação, de forma que a integral representa o caminho óptico (o tempo que a luz leva para viajar uma distância $d$ é $d/c = nd/c_o$. O produto $nd$ é conhecido como caminho óptico).

As trajetórias dos raios são caracterizadas pelas superfícies às quais eles são normais. Seja $S(\mathbf{r})$ uma função potencial cujas superfícies equipotenciais $S(\mathbf{r})=\text{constante}$ são normais aos raios em qualquer ponto $\mathbf{r}$ do espaço. Se a função $S(\mathbf{r})$ é conhecida, as trajetórias dos raios podem ser construídas uma vez que a normal as superfícies equipotenciais em uma posição $\mathbf{r}$ está na direção do vetor gradiente $\vec{∇}S$. A função $S(\mathbf{r})$ é chamada de **eikonal**, similar a função potencial $V(\mathbf{r})$, em eletrostática, onde os raios ópticos fazem o papel do campo elétrico $\mathbf E=-\vec∇$.

Para satisfazer o princípio de Fermat, a eikonal $S(\mathbf r)$ precisa satisfazer uma equação diferencial parcial conhecida como **equação eikonal**

\begin{equation}
\left(\frac{\partial S}{\partial x}\right)^2 + \left(\frac{\partial S}{\partial y}\right)^2 + \left(\frac{\partial S}{\partial z}\right)^2 = n^2
\end{equation}

que usualmente é escrita na forma vetorial:

\begin{equation}
|\vec\nabla S|^2 = n^2
\end{equation}

onde $|\vec\nabla S|^2 = \vec\nabla S\cdot\vec\nabla S$. A prova matemática dessa equação não será apresentada aqui.

*Uma onda é dita paraxial quando as normais às suas frentes de onda são raios paraxiais, ou seja, o campo potencial cujas superfícies equipotenciais descrevem superfícies de mesma fase é uma eikonal. Dessa forma, a direção dos raios é paralela ao vetor gradiente da fase".

Um jeito de se construir uma onda paraxial é começar comm uma onda plana $A\exp(-jkz)$ (usando da representação fasorial de ondas - no caso trata-se de uma onda plana propagando-se na direção $z$) e modular seu envelope complexo como uma função que varia lentamente com a posição no espaço $A(\mathbf r)$, portanto

\begin{equation}
U(\mathbf r) = A(\mathbf r)\exp(-jkz)
\end{equation}

A variação de $A(\mathbf r)$ e sua derivada com a posição z precisam variar lentamente dentro da distância de um comprimento de onda para que a onda mantenha sua característica de onda plana.

Para que a onda paraxial satisfaça a equação de Helmholtz $∇^2U + k^2U=0$, o envelope complexo $A(\mathbf r)$ precisa satisfazer outra equação obtida da substituição da nova expressão de $U$ na equação de Helmholtz. Assumindo-se que $A(\mathbf r)$ varia lentamente com relação a $z$ significa que dentro de uma distância $\Delta << \lambda$, a variação $\Delta A$ é muito menor que $A$ em si $\Delta A << A$. Esta inegualdade de variáveis complexas apelica-se as magnitudes das partes reais e imaginárias separadamente. Uma vez que $\Delta A = (\partial A/\partial z)\Delta z = (\partial A/\partial z) \lambda$, $\partial A/\partial z << A/\lambda  = A k /2\pi$ de modo que

\begin{equation}
\frac{\partial A}{\partial z} << kA
\end{equation}

A derivada $\partial A/\partial z$ em si precisa variar lentmente dentro de uma distância λ, de forma que $\partial ^2 A/\partial z^2 << k\partial A/\partial z$, o que resulta em

\begin{equation}
\frac{\partial^2A}{\partial z^2}<< k^2 A
\end{equation}

Fazendo a substituição de $U$ na equação de Helmholtz e ignorando os temos $\partial^2 A/\partial z^2$ com relação a $k\partial A/\partial z$ ou $k^2 A$, se chega a uma equação diferencial para o envelope complexo $A(\mathbf r)$:

\begin{equation}
∇_T^2 A - j2k\frac{\partial A}{\partial z} = 0
\end{equation}

onde $\partial_T^2 = \partial^2/\partial x^2 + \partial^2/\partial y^2$ é o operador Laplaciano transversal. A equalção acima é a **equação de Hemholtz paraxial**. A solução mais simples para essa equação é a onda paraboloide, que é uma aproximação paraxial de uma onda esférica. Uma outra solução é o **feixe Gaussiano**.

## O feixe Gaussiano

O feixe Gaussiano é uma importante solução da equação de Helmholtz paraxial que exibe características de um feixe óptico. A potência do feixe é principalmente concentrado dentro de um pequeno cilindro que envolve o eixo de feixe. A distribuição de intensidade em qualquer plano transversal é uma função gaussiana de simetria circular centrada em torno do eixo do feixe. A largura dessa função é mínima na cintura do feixe (foco) e se torna graualmente maior a medida que se distancia dessa cintura em ambas as direções do eixo do feixe. As frentes de onda são aproximadamente planas nas proximidades do foco, se curvam gradualmente à medida que se afasta dele e, em maiores distâncias do foco se tornam aproximadamente esféricas. A divergência angular das normais à frente de onda assumem o menor valor permitido pela equação de onda de uma dada largura de feixe. A luz da maioria dos tipos de laser assumem a forma de um feixe gaussiano.

Como mencionado, o feixe Gaussiano é um tipo de onda paraxial. Uma onda paraxial é uma onda plana viaajando ao longo da direção $z$ ($e^{-jkz}$) (com número de onda $k=2\pi/\lambda$ e comprimento de onda $\lambda$), modulado por um envelope complexo $A(\mathbf r)$ que é uma função que varia lentamente com a posição, de forma que sua amplitude complexa é dada por

\begin{equation}
U(\mathbf r)=A(\mathbf r) \exp(-jkz)
\end{equation}

O envelope é tido como aproximadamente constante dentro de uma vizinhança de tamanho $\lambda$, de forma que a onda mantém localmente sua característica de onda plana enquanto exibe normais às frentes de onda que são raios paraxiais.

Para que a amplitude complexa $U(\mathbf r)$ satisfaça a equação de Helmholtz $\nabla^2U + k^2U = 0$, o envelope complexo $A(\mathbf r)$ precisa satisfazer a equação de Helmholtz paraxial

\begin{equation}
\nabla_T^2 A - j2k \frac{\partial A}{\partial z} = 0
\end{equation}

onde $\nabla_T^2=\partial^2/\partial x^2 +\partial ^2/\partial y^2$ é o operador Laplaciano transversal. Uma solução simples para a equação de Helmholtz paraxial é a onda paraboloidal, para a qual

\begin{equation}
A(\mathbf r) = \frac{A_1}{z}\exp\left(-jk\frac{\rho^2}{2z}\right),\space \rho^2=x^2 + y^2
\end{equation}

onde $A_1$ é uma constante.  A onda paraboloidal é uma aproximação paraxial da onda esférica $U(\mathbf r) =  (A_1/r)exp(-jkr)$ quando $x$ e $y$ são muito menores que $z$.

Outra solução da equação de Helmholtz paraxial leva ao feixe Gaussiano. Esta é obitida da onda paraboloidal  pelo uso de uma simples transformação. Uma vez que o envelope complexo da onda paraboloidal é uma solução da eequação de Helmholtz paraxial, também o será uma versão espacialmente deslocada desta, com $z-ξ$ substituindo $z$ onde $ξ$ é uma constante.

\begin{equation}
A(\mathbf r) = \frac{A_1}{q(z)}\exp\left[-jk\frac{\rho^2}{2q(z)}\right],\space q(z) = z-ξ
\end{equation}

Essa solução representa uma onda parabooidal centrada em torno do ponto $z=ξ$ ao invés de $z=0$. A equação acima ainda é uma solução válida mesmo para um $\xi$ complexo, mas a solução adiquire propriedades drasticamente diferentes. Em particular, quando $\xi$ é puramente imaginário, digamos $\xi=-jz_0$ onde $z_0$ é real, a equação apresentada leva ao envelope complexo do feixe Gaussiano

\begin{equation}
A(\mathbf r) = \frac{A_1}{q(z)}\exp\left[-jk\frac{\rho^2}{2q(z)}\right],\space q(z) = z + jz_0
\end{equation}

A quantidade $q(z)$ é chamada de **parâmetro q** do feixe e o parâmetro $z_0$ é conhecido como **alcance de Rayleigh**.

Para separar a amplitude e fase desse envelope complexo, escrevemos a função complexa $1/q(z) = 1/(z+jz_0)$ em termos de suas partes reais e imaginárias definindo-se duas novaz funções reais, $R(z)$ e $W(z)$, de forma que

\begin{equation}
\frac{1}{q(z)} = \frac{1}{R(z)} - j\frac{\lambda}{\pi W^2(z)}
\end{equation}

Substituindo essa expressão no envelope complexo e depois substituindo o envelope complexo na amplitude complexa se chega a

\begin{equation}
U(\mathbf r) = A_0\frac{W_0}{W(z)}\exp\left[-\frac{\rho^2}{W^2(z)}\right]\exp\left[-jkz - jk\frac{\rho^2}{2R(z)} + j\zeta(z)\right]
\end{equation}

sendo

\begin{gather}
W(z) = W_0\sqrt{1 + \left(\frac{z}{z_0}\right)^2}\\
R(z) = z\left[1 + \left(\frac{z_0}{z}\right)^2\right]\\
\zeta(z) = \arctan\frac{z}{z_0}\\
W_0 = \sqrt{\frac{\lambda z_0}{\pi}}
\end{gather}

Uma nova constante $A_0 = A_1/jz_0$ foi adicionada por conveniência.  $W(z)$ é a largura do feixe, $R(z)$ é a curvatura da frente de onda. $W_0$ é a largura do feixe em sua cintura (foco), e $\zeta(z)$ é o excesso de fase de atraso acumulada associada ao **efeito Gouy**. Um parâmetro importante que pode ser obtido das expressões apresentadas é a diverg~encia do feixe, que é comumente o parâmetro apresentado em datasheets além da potência e comprimento de onda. Quando $z >> z_0$:

\begin{equation}
W(z) \approx \frac{W_0}{z_0}z = \theta_0 z.
\end{equation}

O feixe diverge então dentro de um cone de meio ângulo igual a $\theta_0$.

### Feixe Gaussiano de Astigmatismo simples

Diz-se que o feixe gaussiano é simplesmente astigmático quando os parâmetros q do feixe são diferentes entre as direções x e y, o que implica em diferentes propriedades nas duas direções:

\begin{equation}
U(\mathbf r) = U_0(z)\exp\left[-jkz - j\frac{k}{2}\left(\frac{x^2}{q_x(z)} + \frac{y^2}{q_y(z)}\right)\right]
\end{equation}

Para esse caso, é possível mostrar que, quando não há perda da característica de simplesmente astigmático, o problema pode ser separado no plano xz e no plano yz. Assumindo-se como verdadeira essa afirmação, podemos tratar a propagação dos feixes nesses planos separadamente e combiná-los depois para obter-se o feixe real que se propaga através do sistema óptico ($U(\mathbf r) = U_x(\mathbf r)U_y(\mathbf r)$).

Dada a exposição, utilizaremos os resultados apresentados nos capítulos 1, 2 e 3 do livro Fundamentals of Photonics para calcular a intensidade luminosa na área sensível do PPBIO, partindo do pressuposto que todas as etapas do caminho óptico estão alinhadas de forma a produzir o foco desejado. As propriedades usadas nessa dedução constam no livro e nos limitaremos ao uso, sem entrar em detalhes da dedução matemática de tais propriedades.

## Análise do PPBIO usando óptica de feixe por uma aproximação paraxial

A estrutura do PPBIO pode ser decomposta em superfícies que definem lentes, refrações e reflexões. A superfície Sa define a superfície onde ocorrerá a ressonância de plásmons. Superfícies M representam reflexões. Lentes são identificadas com a letra L e refrações com a letra S, sendo excessão desta última a superfície da área sensível Sa. O plano do sensor de imagem é identificado pela letra I.

<div style="text-align: center;">
    <img src="ppbio_sketch.png" alt="Diagrama do PPBIO com definição das superfícies" style="width: 500px;"/>
</div>

Pode-se simplificar a análise lembrando que, ignorando as difrações decorrentes da passagem do feixe por aberturas, as reflexões em planos não alteram as propriedades do feixe, apenas mudam a direção do eixo óptico em relação às coordenadas globais. Podemos então começar pela obtenção dos parâmetros do feixe que chega a área sensível e depois aplicar o traçado de raios para obter o ângulo de incidência do eixo do feixe sobre a superfície Sa. Tomando por referência sempre o eixo do feixe, pode-se representar a estrutura do PPBIO pelo seguinte diagrama:

<div style="text-align: center;">
    <img src="ppbio_plane_diagram.png" alt="Propagação do feixe através do sistema tomando por referência o eixo óptico" style="width: 800px;"/>
</div>

Dado o fato que reflexões em planos (quando a análise tem o eixo do feixe por referência) não alteram as características do feixe, o diagrama pode ser simplificado:

<div style="text-align: center;">
    <img src="ppbio_plane_diagram_simplified.png" alt="Propagação do feixe através do sistema tomando por referência o eixo óptico diagrama simplificado)" style="width: 800px;"/>
</div>

Sejam $f_c, f_f\space e\space n_p$ as distâncias focais das lentes colimadoras Lc1 e Lc2, da lente focalizadora Lf e o índice de refração do substrato óptico do prisma, respectivamente. Podemos tomar duas abordagens:

* Avaliar a propagação considerando as equações que descrevem a transformação dos parâmetros do feixe por uma lente
* Calular o equivalente matricial do sistema óptico e usar a lei ABCD para obter o parâmetro q do feixe de saída do sistema

Nessa primeira etapa estamos preocupados em descrever o feixe incidente na área sensível. O sistema nesse caso vai do laser até a superfície Sa. A primeira das duas abordagens acima se mostra mais vantajosa uma vez que obteremos os parâmetros do feixe diretamente, enquanto na segunda abordagem se obteria o parâmetro q, que por si só não tem significado físico direto.

### Direção paralela a polarização


Começaremos pela propagação do feixe em $x$, tendo em vista que o plano $xz$ é paralelo a polarização do laser e é onde se formará o espectro angular que excitará a ressonância de plásmons de superfície em Sa.

<div style="display: flex; justify-content: center;">
    <img src="beam_focusing.png" alt="Focando um feixe" style="width: 300px;"/>
    <img src="beam_focusing_approx.png" alt="Focando um feixe2" style="width: 300px;"/>
</div>


A transformação de um feixe por uma lente pode ser obtida das equações

\begin{align}
    \text{Cintura do feixe} && W_0' &= M W_0\\
    \text{Posição da cintura} && (z'-f) &= M^2 (z - f)\\
    \text{Profundidade de foco} && 2z_0' &= M^2 (2z_0)\\
    \text{Ângulo de divergência} && 2\theta_0' &= \frac{2\theta_0}{M}\\
    \text{Magnificação} && M &= \frac{M_r}{\sqrt{1 + r^2}}\\
    r = \frac{z_0}{z - f} && M_r &= \left|\frac{f}{z-f}\right|
\end{align}

Seja $\theta_{0x}$ a divergência do laser na direção paralela a polarização e $\lambda$ o comprimento de onda do laser no vácuo:

\begin{align}
W_{0x} &= \frac{\lambda}{\pi\theta_{0x}} \\
z_{0x} &= \frac{\lambda}{\pi\theta_{0x}^2}
\end{align}

Na colimação, colocamos a lente colimadora à uma distância da cintura do feixe do laser igual a distância focal $z = f_c + z_{0x}$ (ver livro texto Exercício 3.2-2):

\begin{align}
    r &=& \frac{z_{0x}}{z - f_c} &=& 1 \\
    M_r &=& \left|\frac{f_c}{z_{0x}}\right|\\
    M &=& \frac{1}{\sqrt 2}\left|\frac{f_c}{z_{0x}}\right|\\
\end{align}

Portanto

\begin{align}
    W_{0xc} &=& \frac{1}{\sqrt 2}\left|\frac{f_c}{z_{0x}}\right| W_{0x}\\
    z_{xc}' &=& \left(\frac{f_c^2}{2z_{0x}^2}\right) z_{0x} + f_c\\ 
    z_{0xc} &=& \left(\frac{f_c^2}{2z_{0x}^2}\right) z_{0x}\\
\end{align}

Agora, colocamos a lente focalizadora sobre a cintura do feixe colimado (ver livro Seção 3.2.B - Beam Focusing):

\begin{align}
W_{0xf} &=& \frac{W_{0xc}}{\sqrt{1 + (z_{0xc}/f_f)^2}} \\
z_{xf}' &=& \frac{f_f}{1 + (f_f/z_{0xc})^2}\\
z_{0xf} &=& \frac{z_{0xc}}{1 + (z_{0xc}/f_f)^2}
\end{align}

Por fim, a refração na base do PPBIO. Em coordenadas do feixe focalizado, a refração, que ocorre a uma distância $d$ da lente focalizadora, ocorrerá em uma posição $z = -z_{xf}' + d$. A refração transforma o parâmetro $q_1$ do feixe incidente em $q_2$ do feixe transmitido por: $q_2 = n_p q_1$.

\begin{gather}
q_{xr} = n_pq_{xf} = n_p(d - z_{xf}) + j n_p z_{0f}\\
\implies z_{xr}' = n_p(z_{xf}' - d)\\
\implies z_{0xr} = n_p z_{0xf}
\end{gather}

por consequência

\begin{equation}
W_{0xr} = \sqrt{\frac{(λ/n_p)z_{0xr}}{π}} = \sqrt{\frac{(λ/n_p)n_p z_{0xf}}{π}} = W_{0xf}
\end{equation}

Para que o foco do feixe refratado ocorra sobre a superfície sensora, $z_{xr}' = d_p$, onde d_p é o caminho óptico percorrido no interior do prisma da base até a área sensível, contando todas as reflexões. Assim

\begin{equation}
d = z_{xf}' - \frac{d_p}{n_p}
\end{equation}

De posse de $z_{0xr}$, basta conhecer o comprimento de onda para que possamos caracterizar o feixe completamente. O comprimento de onda no interior do prisma é $\lambda/n_p$.

#### Calculo dos ângulos de incidência e do campo na área sensível

Como apresentado no início dessa formulação, as direções de propagação são dadas pelas normais às frentes de onda, ou seja, a luz se propaga na direção da normal do gradiente das superfícies equifásicas. Seja a fase do feixe incidente sobre a superfície sensível dada por

\begin{equation}
\varphi_{xr}(z) = kz - \zeta_{xr}(z) + \frac{kx^2}{2R_{xr}(z)}
\end{equation}

O gradiente da fase no plano $xz$ é

\begin{equation}
\vec∇\varphi_{xr}(z) = \frac{kx}{R_{xr}(z)}\hat x + \left[k - \frac{k x^2}{2R_{xr}^2(z)}\left(1 - \frac{z_{0xr}^2}{z^2}\right) - \frac{W_{0xr}^2}{W_{xr}^2(z)}\right]\hat z
\end{equation}

Já o campo elétrico é dado por (ver livro seção 5.4-A: The Gaussian Beam)

\begin{equation}
\mathbf E_{xr}(\mathbf r) = E_0\left(-\hat x + \frac{x}{z + jz_{0xr}}\hat z\right)U_{xr}(\mathbf r)U_{yr}(\mathbf r)
\end{equation}

onde 

\begin{equation}
U_{xr}(\mathbf r) = A_0\frac{W_{0xr}}{W_{xr}(z)}\exp\left[-\frac{x^2}{W_{xr}^2(z)}\right]\exp[-j\varphi_{xr}(z)]
\end{equation}

sendo $k=2\pi n_p/\lambda$. $U_{yr}(\mathbf r)$ ainda será calculado.

Vamos analisar a incidência oblíqua desse feixe focado na superfície sensível. Na imagem a seguir segue uma ilustração do problema:

<div style="text-align: center;">
    <img src="oblique_incidence.png" alt="Incidência oblíqua de um feixe Gaussiano" style="width: 500px;"/>
</div>

Queremos encontrar duas coisas: o campo elétrico incidente sobre Sa e o gradiente da fase (direção de propagação) do campo incidente em pontos de Sa. Da imagem chamaremos o ângulo $o$ de $\theta$. Podemos expressar encontrar $x$ e $z$ em coordenadas de feixe que expressam pontos $s$ sobre a superfície Sa (paralelos ao vetor $\hat s$):

\begin{gather}
x = s\cos\theta\\
z = -s\sin\theta
\end{gather}

Substituindo esse valores nas expressões de gradiente de fase e campo elétrico, podemos descrever a distribuição de intensidade no spot formado na área sensível bem como o espectro angular de incidência e sua distribuição no plano sensível. Para calcular o tamanho do spot na direção s, resolvemos

\begin{equation}
W_{xr}(z = -s\sin\theta) = s \implies W_{0xr}\sqrt{1 + \left(\frac{-s\sin\theta}{z_{0xr}}\right)^2} = s
\end{equation}

Desenvolvendo

\begin{gather}
W_{0xr}^2\left[1 + \frac{s^2\sin^2\theta}{z_{0xr}^2}\right] = s^2\\
\implies \left(\frac{1}{W_{0xr}^2} - \frac{\sin^2\theta}{z_{0xr}^2}\right)s^2 = 1\\
\implies s = \pm \frac{W_{0xr}z_{0xr}}{\sqrt{z_{0xr}^2 - W_{0xr}^2\sin^2\theta}}
\end{gather}

O que significa que o tamanho do spot na direção $x$ na área sensível é:

\begin{equation}
l_x^{\text{spot}} = \frac{2W_{0xr}z_{0xr}}{\sqrt{z_{0xr}^2 - W_{0xr}^2\sin^2\theta}}
\end{equation}

Agora vamos resolver para a direção y, de forma a obter a descrição do campo incidente em duas dimensões.

### Direção perpendicular a polarização

A propagação em $y$ é obtida repetindo-se a análise anterior, mas considerando os posicionamentos dos componentes obtido para $x$ e que a lente focalizadora é cilíndrica e não possui curvatura no plano $yz$.

# General case

## Utils

In [2]:
def Rx(a):
    """ Rotation matrix around the x-axis by an angle a (in radians). """
    return array([
        [1, 0, 0],
        [0, cos(a), -sin(a)],
        [0, sin(a), cos(a)]
    ])

def Ry(b):
    """ Rotation matrix around the y-axis by an angle b (in radians). """
    return array([
        [cos(b), 0, sin(b)],
        [0, 1, 0],
        [-sin(b), 0, cos(b)]
    ])

def Rz(c):
    """ Rotation matrix around the z-axis by an angle c (in radians). """
    return array([
        [cos(c), -sin(c), 0],
        [sin(c), cos(c), 0],
        [0, 0, 1]
    ])

def Rtot(a, b, c):
    """ Combine rotations around the x, y, and z axes. """
    return dot(Rz(c), dot(Ry(b), Rx(a)))

def rot2d(a, b, ang):
    """ Rotation of points on the xy plane around the z-axis """
    return (
        Rz(ang) @ vstack([a.flatten(), b.flatten(), zeros_like(a.flatten())])
    ).reshape((3, *a.shape))[:2, :, :]

In [3]:
import plotly.graph_objects as go

def plot_wavefronts(fig, surfaces):
    for surface in surfaces:
        X,Y,Z = surface
        
        fig.add_trace(go.Surface(
            x=X,
            y=Y,
            z=Z,
            opacity=0.5,
            showscale=False,
            colorscale='reds',
            showlegend=False,
            hoverinfo='none'
        ))

In [4]:
def plot_surface(fig, surface):
    X, Y, Z = surface

    fig.add_trace(go.Surface(
        x=X,
        y=Y,
        z=Z,
        opacity=0.5,
        showscale=False,
        colorscale='greys',
        showlegend=False,
        hoverinfo='none'
    ))    

## Gaussian Beam

In [5]:
class GaussianBeam:
    """
    This class is used to represent a General Astigmatic Gaussian Beam.
    """
    def __init__(self,P=1.,λ=1064e-9,q10=66e-3j,q20=-500e-3 + 266e-3j,θ=(20. + 10.j)*π/180., R=eye(3), O=zeros((3,1)), n=1.):
        self.P = P
        self.λ = λ
        self.q1 = lambda z: z + q10.real + 1.j*q10.imag
        self.q2 = lambda z: z + q20.real + 1.j*q20.imag
        self.θ = θ
        self.R = R
        self.O = O
        self.n = n

        self.k = 2.*π*self.n/λ

        self.Q = lambda z: array([
            [(cos(θ)**2.)/self.q1(z) + (sin(θ)**2.)/self.q2(z),      0.5*sin(2.*θ)*(1./self.q1(z) - 1./self.q2(z))],
            [0.5*sin(2.*θ)*(1./self.q1(z) - 1./self.q2(z)),      (sin(θ)**2.)/self.q1(z) + (cos(θ)**2.)/self.q2(z)]
        ],complex128)
        self.E0 = lambda z: sqrt(P/λ*sqrt(
            4.*self.Q(z)[0,0].imag)*self.Q(z)[1,1].imag - (self.Q(z)[0,1] + self.Q(z)[1,0])**2.
        )
        self.η = lambda z: 0.5*(atan(self.q1(z).real/self.q1(z).imag) + atan(self.q2(z).real/self.q2(z).imag))
        self.E = lambda r,z: self.E0(z)*exp(1.j*self.η(z) - 1.j*(self.k/2.)*(r.T @ self.Q(z) @ r))
    
    @staticmethod
    def simple_astigmatic(P=1., λ=700e-9, θ1=6.*π/180., θ2=12.*π/180., θ=0., R=eye(3), O=zeros((3,1)),n=1.):
        """
        This function is used to redefine the beam as simple astigmatic using the divergence in the x and y directions.
        """
        w10 = λ / (π * θ1)
        w20 = λ / (π * θ2)

        z10 = (π * (w10 ** 2.)) / λ
        z20 = (π * (w20 ** 2.)) / λ

        q10 = 1.j * z10
        q20 = 1.j * z20

        return GaussianBeam(P, λ, q10, q20, θ, R, O, n)

    @staticmethod
    def get_q_parameters_from_Q(Q):
        """
        This static method is used to get the q-parameters and the complex angle θ of a beam given its complex radius
          of curvature tensor Q.
        """
        q1, q2 = 1./linalg.eigvals(Q)
        θ = 0.5*atan((Q[0,1] + Q[1,0]) / (Q[0,0] - Q[1,1]))

        return q1, q2, θ

    def get_elipses_parameters(self,z):
        """
        This function is used to calculate the real parameters needed for intensity and phase ellipses to be drawn
        """

        α = self.θ.real
        β = self.θ.imag
        
        ρ1 = self.q1(z).real/(abs(self.q1(z))**2.)
        ρ2 = self.q2(z).real/(abs(self.q2(z))**2.)

        ω1 = self.q1(z).imag/(abs(self.q1(z))**2.)
        ω2 = self.q2(z).imag/(abs(self.q2(z))**2.)

        return α, β, ρ1, ρ2, ω1, ω2
    
    def intensity_ellipse(self, z):
        """
        This function returns the semi axis and rotation of the intensity elipse.
            
        The intensity elipse (defined by φ_w, w1 and w2 - the angle of rotation and semi-axis) is drawn on the
        plane paralel to xy and containing the informed z point on the axis z, and define the waist shape on the
        corresponding z point.
        """

        α, β, ρ1, ρ2, ω1, ω2 = self.get_elipses_parameters(z)

        φ_w0 = 0.5*arctan2(-(ρ1 - ρ2)*tanh(2.*β),(ω1 - ω2))
        φ_w = unwrap(φ_w0 + α, period=π)

        w1 = 1./sqrt((self.k/4.)*(ω1 + ω2 + sqrt(((ω1 - ω2)**2.)*(cosh(2*β)**2.) + ((ρ1 - ρ2)**2.)*(sinh(2*β)**2.))))
        w2 = 1./sqrt((self.k/4.)*(ω1 + ω2 - sqrt(((ω1 - ω2)**2.)*(cosh(2*β)**2.) + ((ρ1 - ρ2)**2.)*(sinh(2*β)**2.))))

        return φ_w, w1, w2
        
    def phase_ellipse(self, z):
        """
        This function returns the semi axis and rotation of the phase elipse.

        The phase elipse is the elipsoid of curvature matrix:
        C = [[1/R1, 0],[0, 1/R2]]
        on its own coordinate system located at the informed z, with local z equal to global z but axis x and y
        rotated by φ_R around z. This elipsoide is tangent to the point z with concavity directed to the origin of
        the global axis.
        """

        α, β, ρ1, ρ2, ω1, ω2 = self.get_elipses_parameters(z)

        φ_R0 = 0.5*arctan2( (ω1 - ω2)*tanh(2.*β),(ρ1 - ρ2))
        φ_R = unwrap(φ_R0 + α, period=π)

        R1 = 2./(ρ1 + ρ2 + sqrt(((ρ1 - ρ2)**2.)*(cosh(2*β)**2.) + ((ω1 - ω2)**2.)*(sinh(2*β)**2.)))
        R2 = 2./(ρ1 + ρ2 - sqrt(((ρ1 - ρ2)**2.)*(cosh(2*β)**2.) + ((ω1 - ω2)**2.)*(sinh(2*β)**2.)))

        return φ_R, R1, R2
    
    def wavefront_surfaces(self, z, n_points=20, coordinates=None):
        """
        This function uses the intensity and phase elipses to calculate wavefronts within the beam waist as
        surfaces with the curvature given by the phase ellipse data and waist given by the intensity elipse data.
        """
        φ_w, w1, w2 = self.intensity_ellipse(z)
        φ_R, R1, R2 = self.phase_ellipse(z)

        θ = linspace(0., 2*π, n_points)
        ρ = linspace(0., 1., n_points)

        Θ, P = meshgrid(θ, ρ)

        surfaces = []
        for zi, w1i, w2i, φ_wi, R1i, R2i, φ_Ri in zip(z, w1, w2, φ_w, R1, R2, φ_R):
            Xw = w1i * P * cos(Θ)
            Yw = w2i * P * sin(Θ)

            X, Y = rot2d(Xw, Yw, φ_wi)

            XR, YR = rot2d(X, Y, -φ_Ri)
            Z = zi - (0.5 * (XR ** 2.) / R1i + 0.5 * (YR ** 2.) / R2i)

            if isinstance(coordinates, list):
                R = coordinates[0]
                O = coordinates[1]
            elif isinstance(coordinates, str) and (coordinates == "local"):
                R = eye(3)
                O = zeros((3,1))
            else:
                R = self.R
                O = self.O

            X, Y, Z = ((R @ vstack([X.flatten(), Y.flatten(), Z.flatten()])) + O).reshape((3, *X.shape))

            surfaces.append([X,Y,Z])

        return surfaces


In [6]:
gb_ga = GaussianBeam()

z_ga = linspace(-1,1,30)
surfaces_ga = gb_ga.wavefront_surfaces(z_ga)

fig_ga = go.Figure()

plot_wavefronts(fig_ga, surfaces_ga)

fig_ga.update_layout(
    title="Visualização 3D da Cintura e Elipsoide de Curvatura",
    scene=dict(
        xaxis=dict(title='X (m)'),
        yaxis=dict(title='Y (m)'),
        zaxis=dict(title='Z (m)'),
    ),
    autosize=False,
    width=800,
    height=600
)

fig_ga.show()

In [7]:
gb_sa = GaussianBeam.simple_astigmatic(R=Rx(-π/4))

z_sa = linspace(-1e-3, 1e-3, 30)
surfaces_sa = gb_sa.wavefront_surfaces(z_sa)

fig_sa = go.Figure()

plot_wavefronts(fig_sa, surfaces_sa)

fig_sa.update_layout(
    title="Visualização 3D da Cintura e Elipsoide de Curvatura",
    scene=dict(
        xaxis=dict(title='X (m)'),
        yaxis=dict(title='Y (m)'),
        zaxis=dict(title='Z (m)'),
    ),
    autosize=False,
    width=800,
    height=600
)

fig_sa.show()

In [8]:
q1, q2, θ = GaussianBeam.get_q_parameters_from_Q(gb_ga.Q(0))

print(q1)
print(q2)
print(180/π*θ)

(-0+0.066j)
(-0.5000000000000006+0.266j)
(20+10j)


## Surface

In [9]:
class Surface:
    """
    This class models an elipsoidal surface.
    """
    def __init__(self, R=eye(3), O=zeros((3,1)), A=inf, B=inf, C=1., n=1.45):
        self.R = R
        self.O = O
        self.S = diag([1. / (A ** 2.), 1. / (B ** 2.), 1. / (C ** 2.)])
        self.n = n

    def surface(self, n_points=20, xrange=[-1., 1.], yrange=[-1.,1.], zrange=[-1., 1.]):
        """
        This function returns points in global coordinates for plotting the surface.
        """
        cx, cy, cz = sqrt(diag(self.S))
        
        if (cx == 0.) and (cy == 0):
            x = linspace(xrange[0], xrange[1], n_points)
            y = linspace(yrange[0], yrange[1], n_points)
            [X, Y] = meshgrid(x, y)

            Z = - 1./cz * ones(X.shape)
        
        elif (cx == 0):
            B = 1./cy
            C = 1./cz

            θ = linspace(-π/2,π/2, n_points)
            x = linspace(xrange[0], xrange[1], n_points)

            [Θ, X] = meshgrid(θ, x)

            Z = -C * cos(Θ)
            Y = B * sin(Θ)

        elif (cy == 0):
            A = 1./cx
            C = 1./cz

            θ = linspace(-π/2,π/2, n_points)
            y = linspace(xrange[0], xrange[1], n_points)

            [Θ, Y] = meshgrid(θ, y)

            Z = -C * cos(Θ)
            X = A * sin(Θ)
        
        else:
            A, B, C = [1./c for c in [cx, cy, cz]]

            θ = linspace(-π, π, n_points)
            φ = linspace(-π/2, π/2, n_points)
            [Θ, Φ] = meshgrid(θ, φ)

            X = A * cos(Θ) * sin(Φ)
            Y = B * sin(Θ) * sin(Φ)
            Z = -C * cos(Φ)
        
        X, Y, Z = (self.R @ (vstack([X.flatten(), Y.flatten(), Z.flatten()])) + self.O).reshape((3,*X.shape))

        return [X,Y,Z]


    def point_of_incidence(self, gb: GaussianBeam):
        """
        This function calculates the point of incidence of the beam on the surface expressed in (in order) the surface,
        beam and global coordinates.
        """
        k = gb.R @ array([[0],[0],[1]])
        k_prime = self.R.T @ k
        O_prime = self.R.T @ (gb.O - self.O)

        a = k_prime.T @ self.S @ k_prime
        b = 2. * (k_prime.T @ self.S @ O_prime)
        c = O_prime.T @ self.S @ O_prime - 1.

        r = roots(vstack([a,b,c]).flatten())
        t = min(r[r >= 0])

        return [(k_prime * t) + O_prime, gb.R.T @ (k * t - gb.O), (k * t) + gb.O]
        
    def get_local_coordinates(self, Ps):
        """
        This function calculates de othonormal basis of the point of incidence coordinate system expressed 
        in the surface coordinate system. The point of incidence must be expressed in the surface coordinates.
        """

        n = (2 * (self.S @ Ps)).flatten()
        n = n / linalg.norm(n)

        d1 = cross(array([0,1,0]), n)
        d1 = d1 / linalg.norm(d1)

        d2 = cross(n, d1)

        return [d.reshape(3,1) for d in [d1,d2,n]]

    def get_curvature_matrix(self, Ps):
        """
        This function calculates de curature matrix of the surface in a given point of incidence. This curvature matrix is
        expressed in the local coordinates system (coordinate system of the point of incidence). The point of incidence must
        be expressed in surface coordinates.
        """
        M = hstack(self.get_local_coordinates(Ps))

        S = M @ self.S @ M.T

        tx = Ps.flatten()[0]
        ty = Ps.flatten()[1]


        b = (((S[0,2]*tx + S[1,2]*ty)**2.) - S[2,2]*(S[0,0]*(tx**2.) + 2.*S[0,1]*tx*ty + S[1,1]*(ty ** 2.) - 1)) ** (3./2.)
        g = S[0,0]*(S[1,2]**2.) + S[1,1]*(S[0,2]**2.) + S[2,2]*(S[0,1]**2.) - 2.*S[0,1]*S[0,2]*S[1,2] - S[0,0]*S[1,1]*S[2,2]

        return (1/b) * array(
            [[S[0,0]*S[2,2] - (S[0,2]**2.) + g*(ty**2.), S[0,1]*S[2,2] - S[0,2]*S[1,2] - g*tx*ty],
             [S[0,1]*S[2,2] - S[0,2]*S[1,2] - g*tx*ty, S[1,1]*S[2,2] - (S[1,2]**2.) + g*(tx**2.)]]
        )

    def reflect_and_refract(self, gb: GaussianBeam):
        """
        This function calculates the parameters needed to describe the transmitted and reflected beam.
        These parameters are the orthonormal basis of the transmitted and reflected beams and their complex radius of curvature tensor.
        """
        Ps, Pb, Pg = self.point_of_incidence(gb)
        d1, d2, n = self.get_local_coordinates(Ps)
        C = self.get_curvature_matrix(Ps)

        # We need to express beam coordinates in surface coordinates
        xi, yi, zi = [(self.R.T @ gb.R)[:,i].reshape((3, 1)) for i in range(3)]
        Qi = gb.Q(Pb[2,0])

        ni = gb.n
        nt = self.n

        # Law or reflection
        zr = zi - 2 * (zi.T @ n) * n
        
        xr = cross(array([0,1,0]), zr.flatten()).reshape((3,1))
        xr = xr / linalg.norm(xr)
        yr = cross(zr.flatten(), xr.flatten()).reshape((3,1))
        
        # Law of refraction
        ztp = (ni / nt) * (zi - (n.T @ zi) * n)
        zts = - sqrt(1 - (linalg.norm(ztp) ** 2.)) * n
        zt = ztp + zts

        xt = cross(array([0,1,0]), zt.flatten()).reshape((3,1))
        xt = xt / linalg.norm(xt)
        yt = cross(zt.flatten(), xt.flatten()).reshape((3,1))
        
        K = lambda xl, yl: array([[xl.T @ d1, xl.T @ d2],[yl.T @ d1, yl.T @ d2]]).reshape((2,2))

        Ki = K(xi, yi)
        Kr = K(xr, yr)
        Kt = K(xt, yt)

        Qr = linalg.inv(Kr.T) @ (Ki.T @ Qi @ Ki - C * ((n.T @ zi) - (n.T @ zr))) @ linalg.inv(Kr)
        Qt = (ni / nt) * (linalg.inv(Kt.T) @ (Ki.T @ Qi @ Ki - C * ((n.T @ zi) - (nt / ni) * (n.T @ zt))) @ linalg.inv(Kt))

        q1i, q2i, θi = GaussianBeam.get_q_parameters_from_Q(Qi)
        q1r, q2r, θr = GaussianBeam.get_q_parameters_from_Q(Qr)
        q1t, q2t, θt = GaussianBeam.get_q_parameters_from_Q(Qt)

        Ri = self.R @ hstack([xi,yi,zi])
        Rr = self.R @ hstack([xr,yr,zr])
        Rt = self.R @ hstack([xt,yt,zt])
        
        gbi = GaussianBeam(gb.P, gb.λ, q1i, q2i, θi, Ri, Pg, ni)
        gbr = GaussianBeam(gb.P, gb.λ, q1r, q2r, θr, Rr, Pg, ni)
        gbt = GaussianBeam(gb.P, gb.λ, q1t, q2t, θt, Rt, Pg, nt)
        
        return gbi, gbr, gbt


In [10]:
gb = GaussianBeam.simple_astigmatic(R=Rx(-pi/3))
plane = Surface(O=array([[0,0,1 + 1e-12]]).T)

gbi, gbr, gbt = plane.reflect_and_refract(gb)

zi = linspace(-1e-3,0.,30)
zr = linspace(0.,1e-3,30)
zt = linspace(0.,1e-3,20)

surfaces_i = gbi.wavefront_surfaces(zi)
surfaces_r = gbr.wavefront_surfaces(zr)
surfaces_t = gbt.wavefront_surfaces(zt)

fig = go.Figure()

plot_wavefronts(fig, [*surfaces_i, *surfaces_r, *surfaces_t])

surface = plane.surface()
plot_surface(fig, surface)

fig.update_layout(
    title="Visualização 3D da reflexão e refração do feixe focado em um plano",
    scene=dict(
        xaxis=dict(title='X (m)',range=[-0.001, 0.001]),
        yaxis=dict(title='Y (m)',range=[-0.001, 0.001]),
        zaxis=dict(title='Z (m)',range=[-0.001, 0.001]),
    ),
    autosize=False,
    width=800,
    height=600,
)

In [11]:
gb = GaussianBeam.simple_astigmatic()
elipsoid = Surface(A=.5e-2, B=.5e-2, C=.5e-2, O=array([[0,0,1.5e-2]]).T)

gbi, gbr, gbt = elipsoid.reflect_and_refract(gb)

surface_el = elipsoid.surface()

zi = linspace(-1e-2,0, 20)
zr = linspace(0, 2e-2, 40)
zt = linspace(0, 2e-2, 40)

surfaces_i = gbi.wavefront_surfaces(zi)
surfaces_r = gbr.wavefront_surfaces(zr)
surfaces_t = gbt.wavefront_surfaces(zt)

fig_el = go.Figure()

plot_wavefronts(fig_el, [*surfaces_i, *surfaces_t])
plot_surface(fig_el, surface_el)

fig_el.update_layout(
    title="Visualização 3D da reflexão e refração do feixe através de uma superfície esférica",
    scene=dict(
        xaxis=dict(title='X (m)',range=[-1e-2, 1e-2]),
        yaxis=dict(title='Y (m)',range=[-1e-2, 1e-2]),
        zaxis=dict(title='Z (m)',range=[ 0, 2e-2]),
        aspectmode='cube'
    ),
    autosize=False,
    width=800,
    height=600,
)

fig_el.show()